(I'm importing some sklearn features twice, because Google.collab is somehow not seeing them).

In [109]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymystem3 import Mystem
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

Firstly, I load the data and defining the Y matrix, classes. I thought about changing them to an array of 3, but it turned out to be unnecessary with sklearn library.

In [ ]:
data = pd.read_csv('/content/fb_sentiment.csv')
data = data.drop(columns='Unnamed: 0')
data.head()

,FBPost,Label
0,Drug Runners and a U.S. Senator have somethin...,O
1,"Heres a single, to add, to Kindle. Just read t...",O
2,If you tire of Non-Fiction.. Check out http://...,O
3,Ghost of Round Island is supposedly nonfiction.,O
4,Why is Barnes and Nobles version of the Kindle...,N


In [ ]:
Y = data['Label']
Y

(The following function will be usefull when preprocessing posts).

In [ ]:
def remove_duplicates (astring) :
  newstring = astring[0]
  index = 0
  for char in astring:
    if char != newstring[index] :
      newstring += char
      index += 1
  return(newstring)

Secondly, I'm doing the preprocessing. It is a relatively simple one as I tried to eliminate all the punctuation (including the one inside words like *french/american*), links, and names.

In [ ]:
posts = []
for post in data['FBPost'] :
  words = post.strip('.!?,').lower().split()
  sentence = ''
  for word in words :
    word=word.strip(',.!?:;"(){}[]<>\|/-=_£+*&^%$0123456789')
    if '@' in word :
      word = 'MENTION'
    elif 'http' in word or 'www' in word :
      word = 'LINK'
    elif len(word) > 2 :
      for i in range(len(word)-2) :
        if word[i] == word[i+1] and word[i] == word[i+2] :
          word = remove_duplicates(word)
          break
    elif '/' in word :
      word = word.split('/')
      word = word[0]+' '+word[-1]
    elif '!' in word :
      word = word.split('!')
      word = word[0]+' '+word[-1]
    elif '.' in word :
      word = word.split('.')
      word = word[0]+' '+word[-1]
    elif "'" in word :
      word = word.split("'")
      word = word[0]+' '+word[-1]
    sentence += word+' '
  posts.append(sentence.strip())
posts

Also, I lemmatised the text (but I'm sure that this lemmatisation is not perfect because I didn't extract word pairs like *Im* and many more colloquial expressions).

In [ ]:
lemm_posts = []
for post in posts :
  lemm_posts.append(''.join(Mystem().lemmatize(text=post)))
  print(post)
lemm_posts

Thirdly, I'm making a Tf-Idf matrix to work with text data.

In [ ]:
tfidf = TfidfVectorizer(analyzer='word', stop_words=stopwords.words('english'))
posts_tfidf = tfidf.fit_transform(lemm_posts)

Forthly, I divide the sets of data into a train one and a test one.

In [129]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(posts_tfidf, Y, test_size=0.2, random_state=13)

I decided to choose SVM model and MLP model for this task.

In [130]:
SVC_model = sklearn.svm.SVC(kernel='rbf')
SVC_model.fit(x_train, y_train)

SVC()

In [131]:
SVC_prediction = SVC_model.predict(x_test)

In [132]:
mlp = MLPClassifier(hidden_layer_sizes=(5), max_iter=3000)

In [133]:
mlp.fit(x_train, y_train)

MLPClassifier(hidden_layer_sizes=5, max_iter=3000)

In [134]:
MLP_prediction = mlp.predict(x_test)

In [135]:
print(accuracy_score(SVC_prediction, y_test))
print(confusion_matrix(SVC_prediction, y_test))

0.735
[[  0   0   0]
 [  7  16   6]
 [ 14  26 131]]


In [136]:
print(accuracy_score(MLP_prediction, y_test))
print(confusion_matrix(MLP_prediction, y_test))
print(classification_report(MLP_prediction, y_test))

0.73
[[  3   0   1]
 [ 14  34  27]
 [  4   8 109]]
              precision    recall  f1-score   support

           N       0.14      0.75      0.24         4
           O       0.81      0.45      0.58        75
           P       0.80      0.90      0.84       121

    accuracy                           0.73       200
   macro avg       0.58      0.70      0.56       200
weighted avg       0.79      0.73      0.73       200

